## Import libraries

In [1]:
import ee
import geemap
import geemap.chart as ui_chart

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [4]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      Chapter A3.9 Conservation Applications - Assessing the
#                spatial relationship between burned area and precipitation
#  Checkpoint:   A39b
#  Authors:      Harriet Branson, Chelsea Smith
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# ** Upload the area of interest ** #
AOI = ee.Geometry.Polygon([
[
[37.72, -11.22],
[38.49, -11.22],
[38.49, -12.29],
[37.72, -12.29]
]
])
Map.centerObject(AOI, 9)
Map.addLayer(AOI, {
    'color': 'white'
}, 'Area of interest')

# ** MODIS Monthly Burn Area ** #

# Load in the MODIS Monthly Burned Area dataset.
dataset = ee.ImageCollection('MODIS/006/MCD64A1') \
.filter(ee.Filter.date('2010-01-01', '2021-12-31'))

# Select the BurnDate band from the images in the collection.
MODIS_BurnDate = dataset.select('BurnDate')

# A function that will calculate the area of pixels in each image by date.

def func_rlx(img):
    area = ee.Image.pixelArea() \
    .updateMask(
    img
    ) \
    .divide(1e6) \
    .clip(AOI) \
    .reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry = AOI,
        scale = 500,
        bestEffort = True
        ).getNumber(
    'area'
    ); 
    # Add a new band to each image in the collection named area.
    return img.addBands(ee.Image(area).rename('area'))

addArea = func_rlx


















# Apply function on image collection.
burnDateArea = MODIS_BurnDate.map(addArea)

# Select only the area band as we are using system time for date.
burnedArea = burnDateArea.select('area')

# Create a chart that shows the total burned area over time.
burnedAreaChart = \
ui_chart.image_series(
image_collection = burnedArea, 
region = AOI,
reducer = ee.Reducer.mean(),
scale = 500,
x_property = 'system:time_start' 
)
#.setSeriesNames(['Area']) \
burnedAreaChart.set_options({
'title': 'Total monthly area burned in AOI',
'hAxis': {
    'title': 'Date', 
    'format': 'YYYY', 
    'gridlines': {
            'count': 12
        },
    'titleTextStyle': {
            'italic': False,
            'bold': True
        }
},
'vAxis': {
    'title': 'Total burned area (km²)', 
    'maxValue': 2250, 
    'minValue': 0,
    'titleTextStyle': {
            'italic': False,
            'bold': True
        }
},
'lineWidth': 1.5,
'colors': ['d74b46'], 
})
print(burnedAreaChart.getInfo())

# -----------------------------------------------------------------------
# CHECKPOINT
# -----------------------------------------------------------------------

# Load in the CHIRPS rainfall pentad dataset.
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

# Define the temporal range
startyear = 2010
endyear = 2021

# Set the advancing dates from the temporal range.
startdate = ee.Date.fromYMD(startyear, 1, 1)
enddate = ee.Date.fromYMD(endyear, 12, 31)

# Create a list of years
years = ee.List.sequence(startyear, endyear)
# Create a list of months
months = ee.List.sequence(1, 12)

# Filter the dataset based on the temporal range.
Pchirps = chirps.filterDate(startdate, enddate) \
.sort('system:time_start',
False) \
.filterBounds(AOI) \
.select('precipitation'); 

# Calculate the precipitation per month.
def func_laj(y): # Using the list of years based on temporal range.
    def func_mhy(m):
        w = Pchirps.filter(ee.Filter
                .calendarRange(y, y, 'year')) \
            .filter(ee.Filter.calendarRange(m, m,
                'month')) \
            .sum() # Calculating the sum for the month
        return (w.set('year', y) \
            .set('month', m) \
            .set('system:time_start', ee.Date
                .fromYMD(y, m, 1).millis()
            ) # Use millis to keep the system time number.
            .set('date', ee.Date.fromYMD(y, m,
                1)))

    return months.map(func_mhy)

MonthlyRainfall = ee.ImageCollection.fromImages(
    years.map(func_laj).flatten())
# Print the image collection.
print('Monthly Precipitation Image Collection', MonthlyRainfall.getInfo())

# ** Chart: CHIRPS Precipitation ** #

# Create a chart displaying monthly rainfall over a temporal range.
monthlyRainfallChart = \
ui_chart.image_series(
image_collection = MonthlyRainfall.select(
'precipitation'), 
region = AOI,
reducer = ee.Reducer \
.mean(), 
scale = 500,
x_property = 'system:time_start' 
)
#.setSeriesNames(['Precipitation']) \
monthlyRainfallChart.set_options({
'title': 'Total monthly precipitation in AOI', 
'hAxis': {
    'title': 'Date',
    'format': 'YYYY', 
    'gridlines': {
            'count': 12
        },
    'titleTextStyle': {
            'italic': False,
            'bold': True
        }
},
'vAxis': {
    'title': 'Precipitation (mm)', 
    'maxValue': 450, 
    'minValue': 0,
    'titleTextStyle': {
            'italic': False,
            'bold': True
        }
},
'lineWidth': 1.5,
'colors': ['4f5ebd'],
})
print(monthlyRainfallChart.getInfo())

# 2010/2011 wet season total
year = 2010; 
startDate = ee.Date.fromYMD(year, 11, 1); 
endDate = ee.Date.fromYMD(year + 1, 5, 31); 
filtered = chirps \
.filter(ee.Filter.date(startDate, endDate))
Rains10_11Total = filtered.reduce(ee.Reducer.sum()).clip(AOI)

# 2011/2012 wet season total
year = 2011; 
startDate = ee.Date.fromYMD(year, 11, 1); 
endDate = ee.Date.fromYMD(year + 1, 5, 31); 
filtered = chirps \
.filter(ee.Filter.date(startDate, endDate))
Rains11_12Total = filtered.reduce(ee.Reducer.sum()).clip(AOI)

# -----------------------------------------------------------------------
# CHECKPOINT
# -----------------------------------------------------------------------

# ** Combine: CHIRPS Average Rainfall & MODIS Monthly Burn ** #

# Combine the two image collections for joint analysis
bpMerged = burnedArea.merge(MonthlyRainfall)
print('Merged image collection', bpMerged.getInfo())

# ** Chart: CHIRPS Average Rainfall & MODIS Monthly Burn ** #
# Plot the two time series on a graph
bpChart = \
    ui_chart.image_series(**{
        'image_collection': bpMerged, # The merged image collection
        'region': AOI,
        'reducer': ee.Reducer.mean(),
        'scale': 500,
        'x_property': 'system:time_start' # Use system time start for synchronous plotting
    })
    #.setSeriesNames(['Burned Area', 'Precipitation']) # Label series
bpChart.set_chart_type('LineChart') # Define chart type
bpChart.set_options({
        'title': 'Relationship between burned area and rainfall in Chuilexi',
        'interpolateNulls': True, # Interpolate nulls to provide continuous data
        'series': { # Use two sets of series with a target axis to create the two y-axes needed for plotting
            0: { # 0 and 1 reference the vAxes settings below
                'targetAxisIndex': 0,
                'type': 'line',
                'lineWidth': 1.5,
                'color': 'd74b46'
            },
            1: {
                'targetAxisIndex': 1,
                'type': 'line',
                'lineWidth': 1.5,
                'color': '4f5ebd'
            },
        },
        'hAxis': {
            'title': 'Date',
            'format': 'YYYY',
            'gridlines': {
                'count': 12
            },
            'titleTextStyle': {
                'italic': False,
                'bold': True
            }
        },
        'vAxes': {
            0: {
                'title': 'Burned area (km²)', # Label left-hand y-axis
                'baseline': 0,
                'viewWindow': {
                    'min': 0
                },
                'titleTextStyle': {
                    'italic': False,
                    'bold': True
                }
            },
            1: {
                'title': 'Precipitation (mm)', # Label right-hand y-axis
                'baseline': 0,
                'viewWindow': {
                    'min': 0
                },
                'titleTextStyle': {
                    'italic': False,
                    'bold': True
                }
            },
        },
        'curveType': 'function' # For smoothing
    })
bpChart.style().set({
    'position': 'bottom-right',
    'width': '492px',
    'height': '300px'
})

# ** Legend: Rainfall ** #
rain_palette = ['#ffffcc', '#a1dab4', '#41b6c4', '#2c7fb8',
    '#253494'
]

def ColorBar(rain_palette):
    return ui.Thumbnail({
        image: ee.Image.pixelLonLat().select(0),
        params: {
            bbox: [0, 0, 1, 0.1],
            dimensions: '300x15',
            format: 'png',
            min: 0,
            max: 1,
            palette: rain_palette,
        },
        style: {
            stretch: 'horizontal',
            margin: '0px 22px'
        },
    })

def makeRainLegend(lowLine, midLine, highLine, lowText, midText,
    highText, palette):
    labelheader = ui.Label(
        'Total precipitation in wet season (mm)', {
            margin: '5px 17px',
            textAlign: 'center',
            stretch: 'horizontal',
            fontWeight: 'bold'
        });
    labelLines = ui.Panel(
        [
            ui.Label(lowLine, {
                margin: '-4px 21px'
            }),
            ui.Label(midLine, {
                margin: '-4px 0px',
                textAlign: 'center',
                stretch: 'horizontal'
            }),
            ui.Label(highLine, {
                margin: '-4px 21px'
            })
        ],
        ui.Panel.Layout.flow('horizontal'));
    labelPanel = ui.Panel(
        [
            ui.Label(lowText, {
                margin: '0px 14.5px'
            }),
            ui.Label(midText, {
                margin: '0px 0px',
                textAlign: 'center',
                stretch: 'horizontal'
            }),
            ui.Label(highText, {
                margin: '0px 1px'
            })
        ],
        ui.Panel.Layout.flow('horizontal'));
    return ui.Panel({
        widgets: [labelheader, ColorBar(rain_palette),
            labelLines, labelPanel
        ],
        style: {
            position: 'bottom-left'
        }
    });
}
Map.add(makeRainLegend('|', '|', '|', '0', '250', '500', ['#ffffcc',
    '#a1dab4', '#41b6c4', '#2c7fb8', '#253494'
]));

# ** Legend: Burned area ** #
burnLegend = ui.Panel({
    style: {
        position: 'top-left',
        padding: '8px 15px'
    }
});

makeRow = function(color, name) {
    colorBox = ui.Label({
        style: {
            backgroundColor: '#' + color,
            padding: '10px',
            margin: '0 10px 0 0'
        }
    });
    description = ui.Label({
        value: name,
        style: {
            margin: 'o o 6px 6px'
        }
    });
    return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
    });
};

burnPalette = ['FF0000'];
names = ['Burned area'];
for (i = 0; i < 1; i++) {
    burnLegend.add(makeRow(burnPalette[i], names[i]));
}
Map.add(burnLegend);

Map.centerObject(AOI, 9); # Centre the map on the AOI
Map.add(
    bpChart
); # Add the merged burned area & precipitation chart to the map

# ** Chart: Adding an interactive query ** #

# Add a function where if you click on a point in the map it displays the burned area and rainfall for that date
bpChart.onClick(function(xValue, yValue, seriesName) {
    if (!xValue) return;
    # Show layer for date selected on the chart
    equalDate = ee.Filter.equals('system:time_start',
        xValue);
    # Search for the layer in the image collection that links to the selected date
    classificationB = ee.Image(MODIS_BurnDate.filter(
        equalDate).first()).clip(AOI).select('BurnDate');
    classificationR = ee.Image(MonthlyRainfall.filter(
        equalDate).first()).clip(AOI).select(
        'precipitation');
    burnImage = ee.Image(MODIS_BurnDate.filter(equalDate)
        .first());
    date_string = new Date(xValue).toLocaleString(
        'en-EN', {
            dateStyle: 'full'
        });
    rainImage = ee.Image(MonthlyRainfall.filter(equalDate)
        .first());
    date_stringR = new Date(xValue).toLocaleString(
        'en-EN', {
            dateStyle: 'full'
        });
    # Reset the map layers each time a new date is clicked
    Map.layers().reset([classificationB]);
    Map.layers().reset([classificationR]);
    visParamsBurnLayer = { # Visualisation for burned area
        min: 0,
        max: 365,
        palette: ['red']
    };
    visParamsRainLayer = { # Visualisation for rain
        min: 0,
        max: 450,
        palette: ['#ffffcc', '#a1dab4', '#41b6c4',
            '#2c7fb8', '#253494'
        ]
    };
    # Add the layers to the map
    Map.addLayer(classificationR, visParamsRainLayer,
        'Total monthly rainfall on [' + date_string + ']');
    Map.addLayer(classificationB, visParamsBurnLayer,
        'Burned area on [' + date_string + ']');
});

# -----------------------------------------------------------------------
# CHECKPOINT
# -----------------------------------------------------------------------


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 257)

## Display the interactive map

In [ ]:
Map